In [1]:
# IMPORTS
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import vgg16, vgg19
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [2]:
# DATA LOADING

(ds_train, ds_val, ds_test), ds_info = tfds.load(
"tf_flowers", # dataset name
split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
with_info=True,
as_supervised=True
)
num_classes = ds_info.features["label"].num_classes # number of classes
IMG_SIZE = (224,244) # image size

In [3]:
# Preprocessing
def preprocess(image, label):
  image = tf.image.resize(image, IMG_SIZE)
  image = tf.cast(image, tf.float16)
  image = vgg16.preprocess_input(image)  # preprocessing function (depends on VGG)
  return image, label
BATCH = 128
train_ds = ds_train.map(preprocess).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds = ds_val.map(preprocess).batch(BATCH).prefetch(tf.data.AUTOTUNE)

test_ds = ds_test.map(preprocess).batch(BATCH).prefetch(tf.data.AUTOTUNE)

In [4]:
# MODEL CREATION
base_model = vgg16.VGG16(
include_top=False,
weights="imagenet", # pretrained weights
input_shape=IMG_SIZE + (3,)
)
# Freeze or unfreeze layers
for layer in base_model.layers:
  layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Classification head
x = layers.GlobalAveragePooling2D()(base_model.output) # pooling layer
x = layers.Dropout(0.5)(x) # dropout rate
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs=base_model.input, outputs=outputs)

In [6]:

# COMPILATION
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss="sparse_categorical_crossentropy", # loss function
metrics=[ "accuracy" ]
)



In [7]:
# TRAINING
history = model.fit(
train_ds,
validation_data=val_ds,
epochs=5
)

Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.2252 - loss: 7.3398 - val_accuracy: 0.3787 - val_loss: 3.6522
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 389ms/step - accuracy: 0.3610 - loss: 5.2626 - val_accuracy: 0.6594 - val_loss: 1.8100
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 375ms/step - accuracy: 0.5045 - loss: 3.5435 - val_accuracy: 0.7248 - val_loss: 1.1917
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 350ms/step - accuracy: 0.6150 - loss: 2.3844 - val_accuracy: 0.7929 - val_loss: 0.9988
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 327ms/step - accuracy: 0.6657 - loss: 1.9270 - val_accuracy: 0.8120 - val_loss: 0.8883


In [8]:
# EVALUATION
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 625ms/step - accuracy: 0.7775 - loss: 0.9247
Test accuracy: 0.7874659299850464


In [9]:
# MODEL CREATION 2
base_model = vgg16.VGG16(
include_top=False,
weights="imagenet", # pretrained weights
input_shape=IMG_SIZE + (3,)
)
# Freeze or unfreeze layers
for layer in base_model.layers:
  layer.trainable = True

In [10]:

# COMPILATION 2
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
loss="sparse_categorical_crossentropy", # loss function
metrics=[ "accuracy" ]
)



In [11]:
# TRAINING 2
history = model.fit(
train_ds,
validation_data=val_ds,
epochs=5
)

Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 21s 621ms/step - accuracy: 0.6933 - loss: 1.5943 - val_accuracy: 0.8147 - val_loss: 0.8857
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 351ms/step - accuracy: 0.7010 - loss: 1.5609 - val_accuracy: 0.8174 - val_loss: 0.8827
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 425ms/step - accuracy: 0.6987 - loss: 1.6183 - val_accuracy: 0.8174 - val_loss: 0.8680
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 442ms/step - accuracy: 0.7130 - loss: 1.5915 - val_accuracy: 0.8202 - val_loss: 0.8521
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 383ms/step - accuracy: 0.7113 - loss: 1.5566 - val_accuracy: 0.8202 - val_loss: 0.8428


In [12]:
# EVALUATION 2
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - accuracy: 0.7908 - loss: 0.8575
Test accuracy: 0.7983651161193848


In [13]:
# MODEL CREATION 3
base_model = vgg16.VGG16(
include_top=False,
weights="imagenet", # pretrained weights
input_shape=IMG_SIZE + (3,)
)
# Freeze or unfreeze layers
for layer in base_model.layers:
  layer.trainable = False

In [14]:
# Classification head 3
x = layers.GlobalAveragePooling2D()(base_model.output) # pooling layer
x = layers.Dropout(0.7)(x) # dropout rate
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs=base_model.input, outputs=outputs)

In [15]:

# COMPILATION 3
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss="sparse_categorical_crossentropy", # loss function
metrics=[ "accuracy" ]
)



In [16]:
# TRAINING 3
history = model.fit(
train_ds,
validation_data=val_ds,
epochs=10
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 502ms/step - accuracy: 0.1780 - loss: 9.5145 - val_accuracy: 0.2534 - val_loss: 6.1465
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 404ms/step - accuracy: 0.2373 - loss: 8.4410 - val_accuracy: 0.4632 - val_loss: 3.5349
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 341ms/step - accuracy: 0.3353 - loss: 6.7424 - val_accuracy: 0.6485 - val_loss: 1.7381
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 379ms/step - accuracy: 0.4214 - loss: 5.1114 - val_accuracy: 0.7384 - val_loss: 1.3510
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 390ms/step - accuracy: 0.5134 - loss: 3.7692 - val_accuracy: 0.7657 - val_loss: 1.1598
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 430ms/step - accuracy: 0.5989 - loss: 2.9004 - val_accuracy: 0.7902 - val_loss: 1.0287
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 330ms/step - accuracy: 0.6081 - loss: 2.6068 - val_accuracy: 0.8038 - val_loss: 0.9387
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 339ms/step - accuracy: 0.6506 - loss: 2.2337 - val_accuracy:

In [17]:
# EVALUATION 3
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - accuracy: 0.8296 - loss: 0.7052
Test accuracy: 0.8446866273880005


In [4]:
# MODEL CREATION 4
base_model = vgg16.VGG16(
include_top=False,
weights="imagenet", # pretrained weights
input_shape=IMG_SIZE + (3,)
)
# Freeze or unfreeze layers
for layer in base_model.layers:
  layer.trainable = False

In [5]:
# Classification head 4
x = layers.GlobalAveragePooling2D()(base_model.output) # pooling layer
x = layers.Dropout(0.9)(x) # dropout rate
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs=base_model.input, outputs=outputs)

In [6]:
# COMPILATION 4
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss="sparse_categorical_crossentropy", # loss function
metrics=[ "accuracy" ]
)


In [7]:
# TRAINING 4
history = model.fit(
train_ds,
validation_data=val_ds,
epochs=10
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 125s 4s/step - accuracy: 0.2292 - loss: 9.6368 - val_accuracy: 0.3760 - val_loss: 4.1884
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 379ms/step - accuracy: 0.2561 - loss: 9.2287 - val_accuracy: 0.4768 - val_loss: 3.1677
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 376ms/step - accuracy: 0.2835 - loss: 8.8833 - val_accuracy: 0.5613 - val_loss: 2.4929
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 401ms/step - accuracy: 0.3166 - loss: 8.0811 - val_accuracy: 0.6403 - val_loss: 1.9702
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 328ms/step - accuracy: 0.3410 - loss: 7.6915 - val_accuracy: 0.6703 - val_loss: 1.8099
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 343ms/step - accuracy: 0.3750 - loss: 6.7015 - val_accuracy: 0.6948 - val_loss: 1.6552
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 343ms/step - accuracy: 0.4136 - loss: 6.1304 - val_accuracy: 0.7330 - val_loss: 1.4853
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 378ms/step - accuracy: 0.4376 - loss: 5.4335 - val_accuracy:

In [8]:
# EVALUATION 4
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 365ms/step - accuracy: 0.8134 - loss: 0.9098
Test accuracy: 0.8065395355224609


In [4]:
#experiment 5
# MODEL CREATION 5 (Quick Version)
base_model = vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)

# Instead of half, unfreeze just the last block (faster training)
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-4:]:
    layer.trainable = True



In [5]:
# Classification head
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

In [6]:

# COMPILATION 5
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



In [7]:

# TRAINING 5

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5   )


Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 149s 4s/step - accuracy: 0.1851 - loss: 6.9336 - val_accuracy: 0.4605 - val_loss: 1.3991
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 373ms/step - accuracy: 0.5235 - loss: 1.2481 - val_accuracy: 0.7738 - val_loss: 0.6222
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 360ms/step - accuracy: 0.7576 - loss: 0.6848 - val_accuracy: 0.8556 - val_loss: 0.4045
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 343ms/step - accuracy: 0.8576 - loss: 0.3890 - val_accuracy: 0.8774 - val_loss: 0.3565
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 378ms/step - accuracy: 0.9037 - loss: 0.2612 - val_accuracy: 0.8910 - val_loss: 0.3194


In [8]:
# Evaluate
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 708ms/step - accuracy: 0.9148 - loss: 0.2639
Test accuracy: 0.9155
